In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

In [2]:
df = PandasTools.LoadSDF("test_merged.sdf")

In [3]:
df[:2]

,ID,Supplier,clogP,RB,tPSA,Hacc,Hdon,Name,LogSw,Scheme,...,MW_FREE,LogD,BpKa,MPO,rdSMILES,rdMW,rdLogP,rdRotB,rdTPSA,ROMol
0,10000125,ChemBridge,3.250000000000000e+000,1,4.984999999999999e+001,4,0,"rel-(1S,5R)-7-[(2,5-dimethyl-3-thienyl)sulfony...",-3.377000000000000e+000,2826,...,3.305000000000000e+002,,,,Cc1cc(S(=O)(=O)N2C[C@@H]3COC[C@H](C2)N(C)C3)c(...,330.48,1.32,2,49.85,<rdkit.Chem.rdchem.Mol object at 0x7f6040ecfb50>
1,10000301,ChemBridge,2.970000000000000e+000,5,4.941000000000003e+001,2,1,"N-[2-(2,3-dihydro-1H-inden-2-yl)-1,1-dimethyle...",-4.234000000000000e+000,2755,...,3.425000000000000e+002,,,,CCN1CC(C(=O)NC(C)(C)CC2Cc3ccccc3C2)CCC1=O,342.48,2.94,5,49.41,<rdkit.Chem.rdchem.Mol object at 0x7f5eff499150>


## 데이터 타입 전처리

In [4]:
df.dtypes

ID                 object
Supplier           object
clogP              object
RB                 object
tPSA               object
Hacc               object
Hdon               object
Name               object
LogSw              object
Scheme             object
Library            object
StereoChemistry    object
MW                 object
SaltForm           object
MW_FREE            object
LogD               object
BpKa               object
MPO                object
rdSMILES           object
rdMW               object
rdLogP             object
rdRotB             object
rdTPSA             object
ROMol              object
dtype: object

In [5]:
int_cols = []
float_cols = ['rdMW', 'rdLogP', 'rdRotB', 'rdTPSA']

In [6]:
# int_cols에 있는 컬럼들을 int 타입으로 변환
for col in int_cols:
    df[col] = df[col].astype(int)

# float_cols에 있는 컬럼들을 float 타입으로 변환
for col in float_cols:
    df[col] = df[col].astype(float)

In [7]:
df.dtypes

ID                  object
Supplier            object
clogP               object
RB                  object
tPSA                object
Hacc                object
Hdon                object
Name                object
LogSw               object
Scheme              object
Library             object
StereoChemistry     object
MW                  object
SaltForm            object
MW_FREE             object
LogD                object
BpKa                object
MPO                 object
rdSMILES            object
rdMW               float64
rdLogP             float64
rdRotB             float64
rdTPSA             float64
ROMol               object
dtype: object

## 그래프 그리기

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets, HBox, VBox, Layout
from IPython.display import display, clear_output

# 히스토그램 그리는 함수 정의
def plot_histogram(column_name, min_val, max_val, bin_width, aspect_ratio):
    data = df[column_name]
    bins = range(min_val, max_val + bin_width, bin_width)
    
    plt.figure(figsize=(6 * aspect_ratio, 6))
    n, bins, patches = plt.hist(data[(data >= min_val) & (data <= max_val)], bins=bins, edgecolor='black')
    plt.xlabel(column_name)
    plt.ylabel('Count')
    
    # 막대 위에 빈도 수 표시
    for i in range(len(patches)):
        plt.text(patches[i].get_x() + patches[i].get_width() / 2, patches[i].get_height(),
                 str(int(n[i])), ha='center', va='bottom')
    
    # x축 눈금 간격 설정
    plt.xticks(range(min_val, max_val + bin_width, bin_width))
    
    # 그리드 적용
    plt.grid(True)
    
    plt.show()

# int 또는 float 타입의 컬럼만 선택 가능하도록 필터링
numeric_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()

# 위젯 정의
column_name_widget = widgets.Dropdown(
    options=numeric_columns,
    value=numeric_columns[0],
    description='Column:',
    style={'description_width': 'initial'},
    layout=Layout(width='70%')
)

min_val_widget = widgets.IntText(description='Min Value:', style={'description_width': 'initial'}, layout=Layout(width='70%'))
max_val_widget = widgets.IntText(description='Max Value:', style={'description_width': 'initial'}, layout=Layout(width='70%'))
bin_width_widget = widgets.IntText(value=1, description='Bin Width:', style={'description_width': 'initial'}, layout=Layout(width='70%'))
aspect_ratio_widget = widgets.FloatText(value=1.0, description='Aspect Ratio:', style={'description_width': 'initial'}, layout=Layout(width='70%'))

# 컬럼 선택 시 최소값과 최대값을 업데이트하는 함수
def update_min_max(*args):
    column_name = column_name_widget.value
    min_val_widget.value = df[column_name].min()
    max_val_widget.value = df[column_name].max()

# 컬럼 이름 선택 시 트리거
column_name_widget.observe(update_min_max, names='value')

# 초기 값 설정
update_min_max()

# 입력란과 그래프를 각각 왼쪽과 오른쪽에 배치
input_widgets = VBox(
    [column_name_widget, min_val_widget, max_val_widget, bin_width_widget, aspect_ratio_widget]
)

output = widgets.Output()

def update_graph(*args):
    with output:
        clear_output(wait=True)
        plot_histogram(column_name_widget.value, min_val_widget.value, max_val_widget.value, bin_width_widget.value, aspect_ratio_widget.value)

# 각 위젯의 값이 변경될 때마다 그래프 업데이트
for widget in [column_name_widget, min_val_widget, max_val_widget, bin_width_widget, aspect_ratio_widget]:
    widget.observe(update_graph, 'value')

# 초기 그래프 생성
update_graph()

# 입력란과 그래프를 나란히 배치
ui = HBox([input_widgets, output], layout=Layout(align_items='flex-start'))
display(ui)